<a href="https://colab.research.google.com/github/WardaAli-00/PAI-deep-learning/blob/main/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U "tensorflow-text==2.13.*"

  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.36 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
altair 5.5.0 requires typing-extensions>=4.10.0; python_version < "3.14", but you have typing-extensions 4.5.0 which is incompatible.
langchain-core 0.3.25 requires typing-extensions>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
nibabel 5.3.2 requires typing-extensions>=4.6; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatib

### ***Importing Preprocessing Libraries***

In [2]:
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import BertTokenizer
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [3]:
#!pip install --quiet tensorflow_text

import re
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')


stopwords.words('english')
exclude = string.punctuation


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### **Reading Data**

In [4]:
import pandas as pd

# Try reading the files using 'latin-1' encoding
df1 = pd.read_csv('/content/drive/MyDrive/train_data.csv', encoding='latin-1')
df2 = pd.read_csv('/content/drive/MyDrive/test_data.csv', encoding='latin-1')

# Print some rows to confirm
print(df1.head())
print(df2.head())

      id                                           sentence  label
0  12484  I've been buying flowers from this shop for ne...      1
1  47069  The few times I have been to Vegas I like stay...      1
2  56718  The Creative Quest is a terribly cute little s...      1
3  74165  The Paper Studio is a store a gallery and a cl...      1
4  59406  Rabbits make great pets. They're full of chara...      1
      id                                           sentence  label
0  21837  A lot of people think the guy that runs this p...      1
1  66406  Savers. are amazing. A national chain offering...      1
2  63307  There isn't much better than a good Dillard's ...      1
3  79529  Last weekend it was a friend's birthday and th...      1
4  13515  Qcumberz is yet another fun store located on 7...      1


In [5]:
df = pd.concat([df1, df2], axis=0)
print(df.head())

      id                                           sentence  label
0  12484  I've been buying flowers from this shop for ne...      1
1  47069  The few times I have been to Vegas I like stay...      1
2  56718  The Creative Quest is a terribly cute little s...      1
3  74165  The Paper Studio is a store a gallery and a cl...      1
4  59406  Rabbits make great pets. They're full of chara...      1


In [6]:
print(df.columns)

Index(['id', 'sentence', 'label'], dtype='object')


In [7]:
df.isnull().sum()

,0
id,0
sentence,0
label,0


### **Text Cleaning & Preprocessing**

In [8]:
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

#exclude = "!.,?"
def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

In [9]:
df['sentence'] = df['sentence'].str.lower()

In [10]:
df['sentence'] = df['sentence'].fillna('').astype(str)
df['sentence'] = df['sentence'].apply(remove_url)

In [11]:
df['sentence'] = df['sentence'].fillna('').astype(str)
df['sentence'] = df['sentence'].apply(remove_html_tags)

In [12]:
df['sentence'] = df['sentence'].apply(remove_punc)

In [13]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
df['sentence'] = df['sentence'].fillna('').astype(str)
df['sentence'] = df['sentence'].apply(word_tokenize)

In [15]:
from nltk.stem import WordNetLemmatizer

In [16]:
def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(word) for word in text]
    return lemmas

In [17]:
df['sentence'] = df['sentence'].apply(lemmatize_words)

In [18]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [19]:
def remove_stopwords(text):
    new_text = []

    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [20]:
def remove_stopwords(text):
    new_text = [word for word in text if word not in stopwords.words('english')]
    return new_text

In [21]:
df['sentence'] = df['sentence'].apply(remove_stopwords)

In [22]:
df['sentence'] = df['sentence'].astype(str)
df['sentence'] = df['sentence'].apply(lambda x: " ".join(x.split()))

### ***Feature Engineering***

In [23]:
from sklearn.preprocessing import LabelEncoder

X = df['sentence']
Y = df['label']
#print(X)
#print(Y)

encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

X_train,X_test,y_train,y_test = train_test_split(df['sentence'],Y,test_size=0.2,random_state=42)

print(X_train)
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))

1254    ['remember', 'store', 'every', 'street', 'corn...
1561    ['hallmark', 'store', 'located', 'inside', 'ch...
1670    ['amazing', 'service', 'wonderful', 'people', ...
6087    ['great', 'new', 'store', 'summerlin', 'locate...
6669    ['hmmm', 'good', 'shopper', 'around', 'ha', 'e...
                              ...                        
5734    ['typical', 'walmart', 'ill', 'go', 'one', 'ne...
5191    ['dont', 'think', 'could', 'possibly', 'add', ...
5390    ['pretty', 'fun', 'remember', 'dress', 'warm',...
860     ['maybe', 'im', 'type', 'mom', 'know', 'type',...
7270    ['every', 'time', 'go', 'service', 'could', 'b...
Name: sentence, Length: 8000, dtype: object
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [24]:
!pip install transformers torch

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.19.1 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.13.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


# **Fine tuning**

### **Target Column Encoding**

In [25]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import BertTokenizer

### Finetuning using Deep Learning

In [26]:
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4",trainable=True)


text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
#print (text_input)
encoder_inputs = preprocessor(text_input)
# Apply the BERT model
outputs = encoder(encoder_inputs)
# Get pooled output
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
# Add dropout layer for regularization

drop_out = tf.keras.layers.Dropout(0.2,name='dropout')(pooled_output)
# Dense layer for classification
output = tf.keras.layers.Dense(1,activation='sigmoid',name='output')(drop_out)

model=tf.keras.Model(inputs=[text_input],outputs=[output])


In [27]:
#from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(X_train, y_train, epochs=2, validation_split=0.1)

Epoch 1/2
225/225 [==============================] - 10733s 47s/step - loss: 0.7657 - accuracy: 0.5047 - val_loss: 0.6941 - val_accuracy: 0.4512
Epoch 2/2
225/225 [==============================] - 10691s 48s/step - loss: 0.7411 - accuracy: 0.4960 - val_loss: 0.7058 - val_accuracy: 0.5487


### **BAG OF WORDS**

In [29]:
# BAG OF WORDS
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

X_train_bow


<8000x28615 sparse matrix of type '<class 'numpy.int64'>'
	with 475751 stored elements in Compressed Sparse Row format>

In [30]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_bow, y_train)
y_pred_bow = rf_classifier.predict(X_test_bow)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_bow))

              precision    recall  f1-score   support

           0       0.67      0.76      0.71       985
           1       0.73      0.64      0.68      1015

    accuracy                           0.70      2000
   macro avg       0.70      0.70      0.70      2000
weighted avg       0.70      0.70      0.70      2000



### ***Evaluation Metrics:***

In [31]:
from sklearn.metrics import classification_report

# Predictions on the test set
y_pred = model.predict(X_test)

# Convert predictions to binary values (0 or 1)
y_pred_bin = (y_pred > 0.5).astype(int)

# Get classification metrics
print(classification_report(y_test, y_pred_bin))

63/63 [==============================] - 892s 14s/step
              precision    recall  f1-score   support

           0       0.49      1.00      0.66       985
           1       0.00      0.00      0.00      1015

    accuracy                           0.49      2000
   macro avg       0.25      0.50      0.33      2000
weighted avg       0.24      0.49      0.33      2000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### **Metrics for BERT and BoW models**

In [32]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

# Predictions on the test set
y_pred = model.predict(X_test)

# Convert predictions to binary values (0 or 1)
y_pred_bin = (y_pred > 0.5).astype(int)

auc_score = roc_auc_score(y_test, y_pred_bin)  # For binary classification
print(f"ROC-AUC Score: {auc_score}")

63/63 [==============================] - 890s 14s/step
ROC-AUC Score: 0.5


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = model.predict(X_test)
y_pred_bin = (y_pred > 0.5).astype(int)

cm = confusion_matrix(y_test, y_pred_bin)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')

30/63 [=============>................] - ETA: 7:48

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
scores = cross_val_score(rf_classifier, X_train_bow, y_train, cv=5, scoring='f1')
print(f"Cross-validated F1 Scores: {scores}")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()